# Import Packages

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from gurobipy import *
import datetime
import time

# Convert Data to Dataframe

In [2]:
#CONVERT DATA TO DATAFRAME

conn = sqlite3.connect("flight2.db")
c = conn.cursor()
query = c.execute('''SELECT * From flights ''')
cols = [column[0] for column in query.description]
results = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

results.to_csv('flights.csv')

#df = pd.read_sql_query("SELECT * FROM table_name", conn)
#dat = sqlite3.connect("flight.db")
#query = dat.execute("SELECT * From flight")
#cols = [column[0] for column in query.description]
#results= pd.DataFrame.from_records(data = query.fetchall(), columns = cols)

In [3]:
results.head(5)

,price,origin_id,destination_id,outbound_date,inbound_date,currency,origin_city_name,origin_country_name,destination_city_name,destination_country_name,direct,carrier_name,carrier_id,skyscanner_code,date_collected
0,582.0,ATL,PMI,2017-05-01T00:00:00,2017-05-01T00:00:00,USD,Atlanta,United States,Palma,Spain,0,British Airways,881,PMI,2017-04-02 20:18:05.859072
1,118.0,BCN,DUB,2017-05-01T00:00:00,2017-05-01T00:00:00,USD,Barcelona,Spain,Dublin,Ireland,1,Vueling Airlines,7,DUB,2017-04-02 20:18:07.194181
2,54.0,BCN,DUB,2017-05-02T00:00:00,2017-05-02T00:00:00,USD,Barcelona,Spain,Dublin,Ireland,1,Vueling Airlines,7,DUB,2017-04-02 20:18:07.194181
3,48.0,BCN,DUB,2017-05-03T00:00:00,2017-05-03T00:00:00,USD,Barcelona,Spain,Dublin,Ireland,1,Vueling Airlines,7,DUB,2017-04-02 20:18:07.194181
4,38.0,BCN,DUB,2017-05-04T00:00:00,2017-05-04T00:00:00,USD,Barcelona,Spain,Dublin,Ireland,1,Vueling Airlines,7,DUB,2017-04-02 20:18:07.194181


# Convert Dataframe to Dictionary

In [5]:
# create dictionary of dataframes

origins = results['origin_id'].unique()

originsDict = {}
for i in origins:
    originsDict[i] = results[results['origin_id'] == i]

In [6]:
#originsDict

In [7]:
# create dictionary for optimization model
# format: {origin: destination1:{date1:price1, date2:price2},
#                  destination2:{date1:price1, date2:price2}}

X = {}
for i in originsDict.keys():
    X[i] = {}

for i in origins:
    for index, row in originsDict[i].iterrows():
        X[i][row['destination_id']] = {}
    for index, row in originsDict[i].iterrows():
        X[i][row['destination_id']][datetime.datetime.strptime(row['outbound_date'], "%Y-%m-%dT%H:%M:%S").date()]= \
        (datetime.datetime.strptime(row['inbound_date'], "%Y-%m-%dT%H:%M:%S").date(), row['price'])

# Automating the date intervals

In [59]:
user_start = '2017-05-01T00:00:00'
user_end = '2017-05-31T00:00:00'
#reads in the first and last day of the full timeframe of the user

try:
    
    dt_start = datetime.datetime.strptime(user_start, "%Y-%m-%dT%H:%M:%S").date()
    dt_end = datetime.datetime.strptime(user_end, "%Y-%m-%dT%H:%M:%S").date()
    
except ValueError:
    print ("Incorrect format")
    
delta = dt_end - dt_start
numdays = delta.days

date_list = [dt_start + datetime.timedelta(days=x) for x in range(0, numdays)]

#print(date_list[0])
#print(date_list[:5])
#delta.days
#dt_start

# Map date to nodes
#date_list = [dt_start + datetime.timedelta(days=x) for x in range(0, numdays)]
#date_map = {}
#date_map[datetime.date(2017, 5, 1)]

date_list

[datetime.date(2017, 5, 1),
 datetime.date(2017, 5, 2),
 datetime.date(2017, 5, 3),
 datetime.date(2017, 5, 4),
 datetime.date(2017, 5, 5),
 datetime.date(2017, 5, 6),
 datetime.date(2017, 5, 7),
 datetime.date(2017, 5, 8),
 datetime.date(2017, 5, 9),
 datetime.date(2017, 5, 10),
 datetime.date(2017, 5, 11),
 datetime.date(2017, 5, 12),
 datetime.date(2017, 5, 13),
 datetime.date(2017, 5, 14),
 datetime.date(2017, 5, 15),
 datetime.date(2017, 5, 16),
 datetime.date(2017, 5, 17),
 datetime.date(2017, 5, 18),
 datetime.date(2017, 5, 19),
 datetime.date(2017, 5, 20),
 datetime.date(2017, 5, 21),
 datetime.date(2017, 5, 22),
 datetime.date(2017, 5, 23),
 datetime.date(2017, 5, 24),
 datetime.date(2017, 5, 25),
 datetime.date(2017, 5, 26),
 datetime.date(2017, 5, 27),
 datetime.date(2017, 5, 28),
 datetime.date(2017, 5, 29),
 datetime.date(2017, 5, 30)]

# Create list of lists of dates

In [60]:
num_days = 10 #this should be specified by the user

dates = []
dates_list = []
for i in range(0,len(date_list)-num_days+1):
    dates = date_list[0:num_days]
    dates_list.append(dates)
    date_list.pop(0)
    
#dates_list

#All of the following code should be in one big loop
#that iterates through dates_list

[[datetime.date(2017, 5, 1),
  datetime.date(2017, 5, 2),
  datetime.date(2017, 5, 3),
  datetime.date(2017, 5, 4),
  datetime.date(2017, 5, 5),
  datetime.date(2017, 5, 6),
  datetime.date(2017, 5, 7),
  datetime.date(2017, 5, 8),
  datetime.date(2017, 5, 9),
  datetime.date(2017, 5, 10)],
 [datetime.date(2017, 5, 2),
  datetime.date(2017, 5, 3),
  datetime.date(2017, 5, 4),
  datetime.date(2017, 5, 5),
  datetime.date(2017, 5, 6),
  datetime.date(2017, 5, 7),
  datetime.date(2017, 5, 8),
  datetime.date(2017, 5, 9),
  datetime.date(2017, 5, 10),
  datetime.date(2017, 5, 11)],
 [datetime.date(2017, 5, 3),
  datetime.date(2017, 5, 4),
  datetime.date(2017, 5, 5),
  datetime.date(2017, 5, 6),
  datetime.date(2017, 5, 7),
  datetime.date(2017, 5, 8),
  datetime.date(2017, 5, 9),
  datetime.date(2017, 5, 10),
  datetime.date(2017, 5, 11),
  datetime.date(2017, 5, 12)],
 [datetime.date(2017, 5, 4),
  datetime.date(2017, 5, 5),
  datetime.date(2017, 5, 6),
  datetime.date(2017, 5, 7),
  dat

# Create Dummy Dictionary

In [15]:
#create dummy dictionary of cities of interest
#month of May

Xcopy = X.copy()
Y = {}
key_list = origins
#should actually be list that we read in from user input

for i in Xcopy.keys():
    if i in key_list:
        Y[i] = Xcopy[i]

for i in Y.keys():
    for j in Y[i].keys():
        if j not in key_list:
            Y[i][j] = {}
    Y[i] = dict((k, v) for k, v in Y[i].iteritems() if v != {})  


In [61]:
#Y

In [17]:
#create dummy dictionary of dates of interest
Ycopy = Y.copy()
Z = {}
for i in Ycopy.keys():
    Z[i] = {}
    for j in Ycopy[i].keys():
        Z[i][j] = {}
        for k in Ycopy[i][j].keys():
            if k in date_list:
                Z[i][j][k] = Ycopy[i][j][k]
            else:
                Z[i][j][k] = {}
        Z[i][j] = dict((k, v) for k, v in Z[i][j].iteritems() if v != {})

In [62]:
#Z

# Separate out source and non-source dictionaries

In [19]:
# source dictionary

source = 'MCO'

Zcopy = Z.copy()
S = {}

for i in Zcopy.keys():
    if i == source:
        S[source] = Zcopy[source]
    else:
        S[i] = {}

for i in S.keys():
    if source in Zcopy[i].keys():
        S[i][source] = Zcopy[i][source]

S = dict((k, v) for k, v in S.iteritems() if v != {})

In [21]:
# non-source dictionary

Zcopy2 = Z.copy()
Zcopy2[source] = {}
Zcopy2 = dict((k, v) for k, v in Zcopy2.iteritems() if v != {})
#with our current dataset, this will get rid of keys other than the source
#since we are missing so many flights

for i in Zcopy2.keys():
    if source in Zcopy2[i].keys():
        Zcopy2[i][source] = {}
    Zcopy2[i] = dict((k, v) for k, v in Zcopy2[i].iteritems() if v != {})
    
Zcopy2 = dict((k, v) for k, v in Zcopy2.iteritems() if v != {})

# Only one in- and out-bound date for source

In [22]:
dt_start = datetime.date(2017, 5, 1)
dt_end = datetime.date(2017, 5, 30)
#this should come from the iteration:
#for i in dates_list: dt_start = i[0], dt_end = i[-1]

Scopy = S.copy()
T = {}

for i in Scopy.keys():
    T[i] = {}
    for j in Scopy[i].keys():
        T[i][j] = {}

for i in Scopy[source].keys():
    for j in Scopy[source][i].keys():
        if j == dt_start:
            T[source][i][j] = S[source][i][j]

for i in Scopy.keys():
    if i != source:
        for j in Scopy[i][source].keys():
            if j == dt_end:
                T[i][source][j] = Scopy[i][source][j]
                
sourceDict = T

In [23]:
#sourceDict

# Don't include start and end dates in non-source dictionary

In [24]:
nonsourceDict = dict(Zcopy2)

for i in nonsourceDict.keys():
    for j in nonsourceDict[i].keys():
        del nonsourceDict[i][j][dt_start]
        del nonsourceDict[i][j][dt_end]


In [63]:
#nonsourceDict

# Optimization Model

In [247]:
model = Model("WanderWorld")

# Add variables
legs = {} # available legs
cost = {} # corresponding cost
avaiLegsByDest = {} # record all the available legs by Destination
avaiLegsByOri = {} # record all the available legs by Origin
for city in real_cities:
    try:
        cost['ATL', city, 'Source', 0] = X['ATL'][city][date_list[start-1]][1]
        legs['ATL', city, 'Source', 0] = model.addVar(vtype=GRB.BINARY, name="{}_to_{}_day_{}_{}".format(
                        'ATL', city, 'Source', 0))
    except:
        ''
    try:
        cost[city,'ATL',  end-start, 'Sink'] = X[city]['ATL'][date_list[end]][1]
        legs[city,'ATL',  end-start, 'Sink'] = model.addVar(vtype=GRB.BINARY, name="{}_to_{}_day_{}_{}".format(
                        city, 'ATL', (end-start), 'Sink'))
    except:
        ''
    
    for date in real_date_list:
        for destination, flightDic in X[city].iteritems():
            try:
                assert(destination in real_cities)
                cost[city, destination, date_map[date], date_map[flightDic[date][0]]+1] = flightDic[date][1]
                legs[city, destination, date_map[date], date_map[flightDic[date][0]]+1] = \
                model.addVar(vtype=GRB.BINARY, name="{}_to_{}_day_{}_{}".format(
                        city, destination, date_map[date], (date_map[flightDic[date][0]]+1)))
            except:
                ''

for key in legs: # Exclude all the inventory variables
    try:
        avaiLegsByDest[key[1]].append(key)
    except:
        avaiLegsByDest[key[1]] = [key]
        
# Inventory Variables
for date in range(0, end-start):
    for city in real_cities:
        legs[city, city, date, date+1] = \
        model.addVar(vtype=GRB.BINARY, name="{}_Inventory_day_{}_{}".format(
                    city, destination, date, date+1))


        
model.update()

        



In [251]:
print len(legs)
for key in avaiLegsByDest:
    print key
for key in legs:
    print key 

133
STN
LGW
LCY
LHR
LTN
PMI
(u'CDG', u'PMI', 5, 6)
(u'LCY', u'LCY', 0, 1)
(u'PMI', u'PMI', 5, 6)
(u'LCY', u'PMI', 1, 2)
(u'LGW', u'LGW', 1, 2)
(u'LGW', u'PMI', 2, 3)
(u'STN', u'STN', 1, 2)
(u'CDG', u'CDG', 6, 7)
(u'PMI', u'STN', 3, 4)
(u'CDG', u'LGW', 3, 4)
(u'LHR', u'PMI', 0, 1)
(u'PMI', u'STN', 1, 2)
(u'LTN', u'PMI', 0, 1)
(u'CDG', u'PMI', 2, 3)
(u'CDG', u'LGW', 5, 6)
(u'PMI', u'PMI', 6, 7)
(u'LCY', u'LGW', 2, 3)
(u'CDG', u'STN', 4, 5)
(u'LCY', u'PMI', 4, 5)
(u'LGW', u'PMI', 1, 2)
(u'LCY', u'LCY', 3, 4)
(u'CDG', u'LCY', 2, 3)
(u'PMI', u'PMI', 1, 2)
(u'CDG', u'LTN', 0, 1)
(u'LTN', u'LTN', 0, 1)
(u'LGW', u'LGW', 6, 7)
(u'STN', u'STN', 5, 6)
(u'CDG', u'PMI', 4, 5)
(u'LHR', u'PMI', 5, 6)
(u'PMI', u'STN', 2, 3)
(u'LTN', u'PMI', 5, 6)
(u'PMI', u'PMI', 3, 4)
(u'CDG', u'LHR', 0, 1)
(u'LHR', u'PMI', 3, 4)
(u'LTN', u'PMI', 3, 4)
(u'LCY', u'LCY', 4, 5)
(u'PMI', u'PMI', 0, 1)
(u'CDG', u'LCY', 1, 2)
(u'LCY', u'LGW', 5, 6)
(u'LTN', u'LTN', 5, 6)
(u'LGW', u'PMI', 6, 7)
(u'CDG', u'LHR', 2, 3)
(u'CDG

In [252]:
# Set Constraints
UpperBound = 4
LowerBound = 2

# all cities covered and visited only once
for destination in real_cities:
    model.addConstr(quicksum(legs[origin, destination, depart, arri] 
                             for (origin,dest,depart,arri) in avaiLegsByDest[destination])>= 1,
                   "VisitOnlyOnce_{}".format(destination))


# min and max number of days (inventory constraints)
for city in real_cities:
    model.addConstr(quicksum(legs[city, city, startDate, startDate+1] for startDate in range(0,end-start)) <= UpperBound,
                   "Stay_Upper_Bound_{}".format(city))
    model.addConstr(quicksum(legs[city, city, startDate, startDate+1] for startDate in range(0,end-start)) >= LowerBound,
                   "Stay_Lowe_rBound_{}".format(city))

# Flow Balance
avaiLegsByDest = {} # record all the available legs by Destination
avaiLegsByOri = {} # record all the available legs by Origin
for (origin, destination, depart, arri) in legs: # With Inventory Variables
    try:
        avaiLegsByDest[destination,arri].append((origin, destination, depart, arri))
    except:
        avaiLegsByDest[destination,arri] = [(origin, destination, depart, arri)]
    try:
        avaiLegsByOri[origin,depart].append((origin, destination, depart, arri))
    except:
        avaiLegsByOri[origin,depart] = [(origin, destination, depart, arri)]

for city in real_cities:
    for date in range(0,end-start+1):
        if(date==0):
            model.addConstr(legs['ATL', city, 'Source', 0] == 
                            quicksum(legs[origin, destination, depart, arri] 
                                     for (origin, destination, depart, arri) in avaiLegsByOri[city,date]),
                                    "FlowBalance_{}_day{}".format(city,date))
        elif(date==end-start):
            model.addConstr(legs[city, 'ATL', end-start, 'Sink'] == 
                            quicksum(legs[origin, destination, depart, arri] 
                                     for (origin, destination, depart, arri) in avaiLegsByDest[city,date]),
                                    "FlowBalance_{}_day{}".format(city,date))
        else:
            model.addConstr(quicksum(legs[origin, destination, depart, arri] 
                                     for (origin, destination, depart, arri) in avaiLegsByOri[city,date]) == 
                            quicksum(legs[origin, destination, depart, arri] 
                                     for (origin, destination, depart, arri) in avaiLegsByDest[city,date]),
                                    "FlowBalance_{}_day{}".format(city,date))

model.update()

KeyError: ('ATL', u'CDG', 'Source', 0)

In [237]:
# Set Objects
obj = quicksum(cost[key]*legs[key] for key in cost)

model.setObjective(obj, GRB.MINIMIZE)

model.update()

model.optimize()

Optimize a model with 77 rows, 147 columns and 469 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e+01, 1e+03]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 4e+00]
Presolve removed 28 rows and 58 columns
Presolve time: 0.00s
Presolved: 49 rows, 89 columns, 299 nonzeros
Variable types: 0 continuous, 89 integer (89 binary)
Found heuristic solution: objective 449.0000000

Root relaxation: objective 9.897143e+01, 51 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   98.97143    0   36  449.00000   98.97143  78.0%     -    0s
     0     0  149.34483    0   33  449.00000  149.34483  66.7%     -    0s
H    0     0                     333.0000000  149.34483  55.2%     -    0s
H    0     0                     175.0000000  149.34483  14.7%     -    0s
*    0     0               0     166.0000000  166.00000

In [238]:
for v in model.getVars():
    print v.varName, v.x

print 'Obj:', model.objVal

ATL_to_CDG_day_Source_0 1.0
CDG_to_ATL_day_7_Sink 0.0
CDG_to_STN_day_0_1 0.0
CDG_to_PMI_day_0_1 0.0
CDG_to_LGW_day_0_1 0.0
CDG_to_LHR_day_0_1 0.0
CDG_to_LCY_day_0_1 0.0
CDG_to_LTN_day_0_1 0.0
CDG_to_STN_day_1_2 0.0
CDG_to_PMI_day_1_2 0.0
CDG_to_LGW_day_1_2 0.0
CDG_to_LHR_day_1_2 0.0
CDG_to_LCY_day_1_2 0.0
CDG_to_LTN_day_1_2 0.0
CDG_to_STN_day_2_3 0.0
CDG_to_PMI_day_2_3 -0.0
CDG_to_LGW_day_2_3 -0.0
CDG_to_LHR_day_2_3 -0.0
CDG_to_LCY_day_2_3 -0.0
CDG_to_LTN_day_2_3 -0.0
CDG_to_STN_day_3_4 -0.0
CDG_to_PMI_day_3_4 -0.0
CDG_to_LGW_day_3_4 -0.0
CDG_to_LHR_day_3_4 0.0
CDG_to_LCY_day_3_4 -0.0
CDG_to_LTN_day_3_4 0.0
CDG_to_STN_day_4_5 0.0
CDG_to_PMI_day_4_5 -0.0
CDG_to_LGW_day_4_5 -0.0
CDG_to_LHR_day_4_5 1.0
CDG_to_LCY_day_4_5 -0.0
CDG_to_LTN_day_4_5 -0.0
CDG_to_STN_day_5_6 0.0
CDG_to_PMI_day_5_6 0.0
CDG_to_LGW_day_5_6 0.0
CDG_to_LHR_day_5_6 0.0
CDG_to_LCY_day_5_6 0.0
CDG_to_LTN_day_5_6 0.0
CDG_to_STN_day_6_7 0.0
CDG_to_PMI_day_6_7 0.0
CDG_to_LGW_day_6_7 0.0
CDG_to_LHR_day_6_7 0.0
CDG_to_LCY_da

In [168]:
print len(legs)
print len(availableLegs)
print len(cost)
#print legs
#print cost


98
84
84


In [ ]:
# Add constraints


In [ ]:
, name="{}_to_{}_day{}_to_{}".format(  
                    origin,destination,date_map(datePair[0]), date_map(datePair[1]))

In [ ]:
model.addVar(vtype=GRB.BINARY)

In [64]:
def get_items(dict_object):
    for key in dict_object:
        yield key, dict_object[key]
for a, b in X.iteritems:
    print a,b

TypeError: 'builtin_function_or_method' object is not iterable

In [ ]:
#GUROBI OPTIMIZATION MODEL

try:

    # Create a new model
    m = Model("WanderWorld")

    # Create variables
    
    # Whether to buy flight arc (i,j)
    for i, j, k:
    X[i][j][k] = m.addVar(vtype=GRB.BINARY, name="leg")
    #NOTE: i and j are codes such as "ATL", NOT indices; k is an index
    # i = origin, j = destination, k = flight (date and price)
    
    # Integrate new variables
    m.update()

    # Set objective
    # Minimize cost
    c = X[i][j][k][2]
    m.setObjective(quicksum(c*x for ?), GRB.MINIMIZE)

    # Add constraints
    
    # all cities covered and visited only once
    m.addConstr(quicksum(X[i][j][k] for i for k) == 1) for j
    
    # min and max number of days (inventory constraints)
    m.addConstr(quicksum(X[i][i][k] for k) == 1) for i
    # need to add to our database/dictionary that the first destination for each origin
    # is itself with a flight on each day, each costing $0
    
    # choose subset / partition of cities

    # optimize
    m.optimize()

    for v in m.getVars():
        print v.varName, v.x

    print 'Obj:', m.objVal

except GurobiError:
    print 'Error reported'

